# Tokenization

In [1]:
import MeCab
import re
import pandas as pd
import numpy as np

mecab = MeCab.Tagger()

한국어 형태소 분석기인 mecab을 가지고 Tokenization을 해주었습니다. 명사만 따로 추출하여 wordcloud로 확인하기 위해서 명사를 따로 추출했고 토큰화가 완료된 문서들을 str_token의 형태로 추가했습니다.

In [38]:
df_ = pd.read_csv('./data/crawling/raw/df_complete.csv',index_col = 0)
del1 = df_[df_.rmk != '정'] #rmk 제거
df = del1[del1.rmk != '정연'].reset_index(drop=True)
df.rmk = df.rmk.fillna('nan')

In [54]:
#display(df.str[df.str.isnull() != False])
drop_ls = [x for x in df.str[df.str.isnull() != False].index]
df = df.drop(drop_ls, axis=0)

In [58]:
def mecab_morphs(text):   # konlpy의 mecab output 형태와 같게 만들어주는 함수
    morphs = []
    
    pattern = re.compile(".*\t[A-Z]+") 
    
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    for token in temp:
        morphs.append(token[0])
    
    return morphs

In [59]:
def mecab_nouns(text):#명사만 반환
    morphs = []
    pattern = re.compile(".*\t[A-Z]+")
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
    for token in temp:
        if token[1] == 'NNG':
            morphs.append(token[0])
    return morphs

In [60]:
def preprocessing(document): #특수문자 기본 전처리 함수
    docu = []
    for doc in document:
        doc = re.sub('[^가-힣0-9\s.]','',doc)
        doc = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',doc)
        doc = re.sub('[0-9]+','num',doc) #숫자를 special token num으로 치환
        pattern = re.compile(r'\s+') #중복띄어쓰기 제거
        doc = re.sub(pattern,' ',doc)
        doc = doc.replace('\xa0','') 
        doc = doc.replace('\n','')
        docu.append(doc)

    return docu

In [61]:
str_ls = [x for x in df['str']]
corrected_ls = preprocessing(str_ls)
df['str'] = corrected_ls

In [62]:
kk = [mecab_morphs(x) for x in df['str']]
kl = [] #리스트안에 형태소분석된 토큰들이 들어있는 형태라서 str로 바꿔서 새 컬럼에 추가해준다
for k in kk:
    kl.append(' '.join(k))
df['str_tokens'] = kl
df = df.loc[:,('crp_nm','rpt_nm','rcp_no','rcp_dt','rmk','str','str_tokens')]

nouns_ls = [mecab_nouns(x) for x in df['str']] #noun wordcloud용
ke=[]
for k in nouns_ls:
    ke.append(' '.join(k))
df['str_nouns'] = ke

In [ ]:
rcp_ls = df['rcp_no']  #연도 column 생성
rcp_ls = list(map(str,rcp_ls))
rcp__ls = []
for i in rcp_ls:
    rcp__ls.append(int(i[0:4]))
df.insert(0,'연도',rcp__ls) 

ls = []
for i in df.연도:
    ls.append(str(i))
title_ls = ls+df.crp_nm

df_cosine =  df.copy()
df_cosine.insert(0,'title',title_ls)  #새로운 title column 생성
df_cosine = df_cosine.reset_index(drop=True)

In [69]:
df_cosine.head()

,title,연도,crp_nm,rpt_nm,rcp_no,rcp_dt,rmk,str,str_tokens,str_nouns
0,2019동화약품,2019,동화약품,사업보고서 (2018.12),2.019040e+13,20190401.0,연,. 이사의 경영진단 및 분석의견num. 예측정보에 대한 주의사항당사가 본 사업보고서...,. 이사 의 경영 진단 및 분석 의견 num . 예측 정보 에 대한 주 의 사항 당...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 사업 보고서 미래 발생 예상 예...
1,2018동화약품,2018,동화약품,[기재정정]사업보고서 (2017.12),2.018062e+13,20180618.0,nan,. 이사의 경영진단 및 분석의견num. 예측정보에 대한 주의사항당사가 본 사업보고서...,. 이사 의 경영 진단 및 분석 의견 num . 예측 정보 에 대한 주 의 사항 당...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 사업 보고서 미래 발생 예상 예...
2,2017동화약품,2017,동화약품,[기재정정]사업보고서 (2016.12),2.017041e+13,20170405.0,nan,. 이사의 경영진단 및 분석의견num. 예측정보에 대한 주의사항당사가 본 사업보고서...,. 이사 의 경영 진단 및 분석 의견 num . 예측 정보 에 대한 주 의 사항 당...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 사업 보고서 미래 발생 예상 예...
3,2016동화약품,2016,동화약품,사업보고서 (2015.12),2.016033e+13,20160330.0,nan,. 이사의 경영진단 및 분석의견num. 예측정보에 대한 주의사항당사가 본 사업보고서...,. 이사 의 경영 진단 및 분석 의견 num . 예측 정보 에 대한 주 의 사항 당...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 사업 보고서 미래 발생 예상 예...
4,2015동화약품,2015,동화약품,사업보고서 (2014.12),2.015033e+13,20150331.0,nan,. 이사의 경영진단 및 분석의견num. 예측정보에 대한 주의사항당사가 본 사업보고서...,. 이사 의 경영 진단 및 분석 의견 num . 예측 정보 에 대한 주 의 사항 당...,이사 경영 진단 분석 의견 예측 정보 주 사항 당사 사업 보고서 미래 발생 예상 예...


In [72]:
len(df_cosine.crp_nm.unique())

275